In [1]:
from dotenv import load_dotenv
import os
import psycopg2 as psql
import pprint
import numpy as np
import requests
from pprint import pprint
from steam_web_api import Steam
import pandas as pd


In [2]:
load_dotenv()

KEY=os.getenv('STEAM_API_KEY')
User=os.getenv('User')
Password=os.getenv('Password')
Host=os.getenv('Host')
Database=os.getenv('Database')
Port=os.getenv('Port')
steam = Steam(KEY)

In [20]:
conn = psql.connect(
    database=Database,
    user=User,
    host=Host,
    password=Password,
    port=Port
)

cur=conn.cursor()

try:
    sql="""
    CREATE TABLE student.de10ATCapstoneProject (
    id INT GENERATED ALWAYS AS IDENTITY,
    user_name VARCHAR(60) NOT NULL,
    steam_id BIGINT NOT NULL,
    avatar_url VARCHAR(255) NOT NULL);
    """
    cur.execute(sql)
except:
    print("This table already exists")
    sql="""
    SELECT *
    FROM student.de10ATCapstoneProject;
    """ 
    cur.execute(sql)

    rows=cur.fetchall()

    for row in rows:
        print(row)


cur.close()

This table already exists


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [39]:
def get_details(userName):
    try:
        userDetails=steam.users.search_user(userName)
        if userDetails == 'No match':
            print("There is no user with this name")
        else:
            return userDetails['player']['personaname'], userDetails['player']['steamid'],userDetails['player']['avatarfull'],
    except:
        return 'Error', np.NAN, np.NAN, 

#pprint(get_details('SuperAbz'))
steam.users.search_user('SuperAbz')

{'player': {'steamid': '76561198341402951',
  'communityvisibilitystate': 3,
  'profilestate': 1,
  'personaname': 'SuperAbz',
  'commentpermission': 2,
  'profileurl': 'https://steamcommunity.com/id/SuperAbz/',
  'avatar': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9.jpg',
  'avatarmedium': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9_medium.jpg',
  'avatarfull': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9_full.jpg',
  'avatarhash': '9460ad139083afc82c9025cb03ab9a51bd876dd9',
  'lastlogoff': 1717937496,
  'personastate': 0,
  'realname': 'TheLastDodo',
  'primaryclanid': '103582791429521408',
  'timecreated': 1477768490,
  'personastateflags': 0,
  'loccountrycode': 'GB'}}

In [15]:
UserData=get_details('SuperAbz')

cur=conn.cursor()

newData = {
    'user_name': UserData[0],
    'steam_id': UserData[1],
    'avatar_url': UserData[2]
}

print(newData)

sql = """
INSERT INTO student.de10_at_capstone_project1 (user_name, steam_id, avatar_url)
VALUES (%s, %s, %s)
"""

# Execute the query with parameters
cur.execute(sql, (
    newData['user_name'],
    newData['steam_id'],
    newData['avatar_url']
))
conn.commit()
cur.close()
conn.close()

{'user_name': 'SuperAbz', 'steam_id': '76561198341402951', 'avatar_url': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9_full.jpg'}


NumericValueOutOfRange: value "76561198341402951" is out of range for type integer
LINE 3: VALUES ('SuperAbz', '76561198341402951', 'https://avatars.st...
                            ^


In [60]:
sql="""
SELECT *
FROM student.de10_at_capstone_project1 


"""
cur.execute(sql)
rows=cur.fetchall()
print(rows)


# if rows:    
#     for row in rows:
#         print(row)
# else:
#     print("This user is not in the table")

[]


In [39]:
steam = Steam(KEY)
#response = requests.get(steam)
print(steam)
userDetails=steam.users.search_user("SuperAbz")
pprint(userDetails)
#name=steam.users.search_user("SuperAbz")['player']['personaname']
#print(userDetails['player']['personaname'])


{'player': {'avatar': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9.jpg',
            'avatarfull': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9_full.jpg',
            'avatarhash': '9460ad139083afc82c9025cb03ab9a51bd876dd9',
            'avatarmedium': 'https://avatars.steamstatic.com/9460ad139083afc82c9025cb03ab9a51bd876dd9_medium.jpg',
            'commentpermission': 2,
            'communityvisibilitystate': 3,
            'lastlogoff': 1717937496,
            'loccountrycode': 'GB',
            'personaname': 'SuperAbz',
            'personastate': 0,
            'personastateflags': 0,
            'primaryclanid': '103582791429521408',
            'profilestate': 1,
            'profileurl': 'https://steamcommunity.com/id/SuperAbz/',
            'realname': 'TheLastDodo',
            'steamid': '76561198341402951',
            'timecreated': 1477768490}}


In [53]:
user = steam.users.get_account_public_info(76561198341402951)
print(user)

{'response': {}}


In [19]:
games = steam.apps.search_games("terr")
pprint(games['apps'][0][img])
pprint(games['apps'][0][link])

{'id': [105600],
 'img': 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/105600/capsule_sm_120.jpg?t=1666290860',
 'link': 'https://store.steampowered.com/app/105600/Terraria/?snr=1_7_15__13',
 'name': 'Terraria',
 'price': '$4.99'}


In [34]:
user = steam.users.get_user_recently_played_games("76561198341402951")
print(user)

{'total_count': 0}


In [36]:
games = steam.users.get_owned_games("76561198341402951")
#pprint(games)
sorted_games = sorted(games['games'], key=lambda x: x['playtime_forever'], reverse=True)
gamesList = []
for game in sorted_games[:5]:
    game_info = steam.apps.search_games(game['name'])
    if game_info['apps']:
        game_data = game_info['apps'][0]
        game_details = {
            'gamesImg': game_data['img'],
            'gameLink': game_data['link'],
            'gameName': game['name'],
            'Playtime': game['playtime_forever']
        }
        gamesList.append(game_details)

pprint(gamesList)
#https://cdn.cloudflare.steamstatic.com/steamcommunity/public/images/apps/400/cfa928ab4119dd137e50d728e8fe703e4e970aff.jpg
#https://cdn.cloudflare.steamstatic.com/steamcommunity/public/images/apps/4000/cfa928ab4119dd137e50d728e8fe703e4e970aff.jpg

[{'Playtime': 7015,
  'gameLink': 'https://store.steampowered.com/app/397540/Borderlands_3/?snr=1_7_15__13',
  'gameName': 'Borderlands 3',
  'gamesImg': 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/397540/capsule_sm_120.jpg?t=1693524259'},
 {'Playtime': 2844,
  'gameLink': 'https://store.steampowered.com/app/330830/Tales_from_the_Borderlands/?snr=1_7_15__13',
  'gameName': 'Tales from the Borderlands',
  'gamesImg': 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/330830/capsule_sm_120.jpg?t=1693524232'},
 {'Playtime': 2200,
  'gameLink': 'https://store.steampowered.com/app/49520/Borderlands_2/?snr=1_7_15__13',
  'gameName': 'Borderlands 2',
  'gamesImg': 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/49520/capsule_sm_120.jpg?t=1693524237'},
 {'Playtime': 1855,
  'gameLink': 'https://store.steampowered.com/app/22380/Fallout_New_Vegas/?snr=1_7_15__13',
  'gameName': 'Fallout: New Vegas',
  'gamesImg': 'https://shared.akamai

In [60]:
def getUserDetails(userName):
    try:
        userDetails=steam.users.search_user(userName)
        if userDetails== 'No match':
            print("There is no user with this name")
        else:
            steamId=userDetails['player']['steamid']
            ##print(steamId)
            gamesDetails = steam.users.get_owned_games(steamId)
            ##pprint(gamesDetails)
            sorted_games = sorted(gamesDetails['games'], key=lambda x: x['playtime_forever'], reverse=True)
            gamesList = []
            for game in sorted_games[:5]:
                game_info = steam.apps.search_games(game['name'])
                if game_info['apps']:
                    game_data = game_info['apps'][0]
                    game_details = {
                        'gamesImg': game_data['img'],
                        'gameLink': game_data['link'],
                        'gameName': game['name'],
                        'Playtime': game['playtime_forever']
                    }
                    gamesList.append(game_details)


            return userDetails['player']['personaname'], userDetails['player']['steamid'],userDetails['player']['avatarfull'],gamesList[0]['gameName'],gamesList[0]['gamesImg'],gamesList[0]['gameLink'],gamesList[0]['Playtime'],gamesList[1]['gameName'],gamesList[1]['gamesImg'],gamesList[1]['gameLink'],gamesList[1]['Playtime'],gamesList[2]['gameName'],gamesList[2]['gamesImg'],gamesList[2]['gameLink'],gamesList[2]['Playtime']
    except AttributeError:
        return 'Error', np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN , np.NAN, np.NAN, np.NAN, np.NAN , np.NAN, np.NAN, np.NAN, np.NAN 
    

pprint(getUserDetails('Deejayah'))


('Slightly Oversized Cat',
 '76561198045983134',
 'https://avatars.steamstatic.com/b3877044cdfff2ddb4288f670e0656064d9f1b2d_full.jpg',
 "Sid Meier's Civilization V",
 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/8930/capsule_sm_120.jpg?t=1579731804',
 'https://store.steampowered.com/app/8930/Sid_Meiers_Civilization_V/?snr=1_7_15__13',
 74755,
 'Counter-Strike 2',
 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/730/capsule_sm_120.jpg?t=1719426374',
 'https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_15__13',
 54758,
 'Terraria',
 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/105600/capsule_sm_120.jpg?t=1666290860',
 'https://store.steampowered.com/app/105600/Terraria/?snr=1_7_15__13',
 50636)


In [58]:
conn = psql.connect(
    database=Database,
    user=User,
    host=Host,
    password=Password,
    port=Port
)

cur=conn.cursor()

userData=getUserDetails('shadeisalive')

sql = """
INSERT INTO student.de10ATCapstoneProject (user_name, steam_id, avatar_url, game_name1,game_img1,game_url1,playtime1,game_name2, game_img2, game_url2, playtime2,game_name3, game_img3, game_url3, playtime3)
VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)
"""

# Execute the query with parameters
cur.execute(sql, (
    userData[0],
    userData[1],
    userData[2],
    userData[3],
    userData[4],
    userData[5],
    userData[6],
    userData[7],
    userData[8],
    userData[9],
    userData[10],
    userData[11],
    userData[12],
    userData[13],
    userData[14]
))
conn.commit()


cur.close()
conn.close()